In [1]:
import gradio as gr
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
from sympy.printing.octave import octave_code
import os

# ==============================================================================
# 1. CLASSES DE BACK-END (MESMA LÓGICA DO EXECUTÁVEL)
# ==============================================================================

# --- CLASSE BASE (AR / SECO) ---
class RobotMathEngine:
    def __init__(self, joint_config, link_vectors_mask):
        self.joint_config = joint_config
        self.link_vectors_mask = [sp.Matrix(v) for v in link_vectors_mask]

        self.t = sp.symbols('t')
        self.g = sp.symbols('g')
        self.q, self.dq, self.params_list = [], [], []
        self.frames, self.rotation_matrices = [], []
        self.com_positions_global, self.angular_velocities = [], []
        self.masses = []

        self.M, self.G_vec, self.C_total, self.Jacobian = None, None, None, None

    def _rot_matrix_local(self, axis, angle):
        c, s = sp.cos(angle), sp.sin(angle)
        if axis == 'x': return sp.Matrix([[1,0,0],[0,c,-s],[0,s,c]])
        if axis == 'y': return sp.Matrix([[c,0,s],[0,1,0],[-s,0,c]])
        if axis == 'z': return sp.Matrix([[c,-s,0],[s,c,0],[0,0,1]])
        return sp.eye(3)

    def _get_axis_vector(self, axis):
        if axis == 'x': return sp.Matrix([1, 0, 0])
        if axis == 'y': return sp.Matrix([0, 1, 0])
        if axis == 'z': return sp.Matrix([0, 0, 1])
        return sp.Matrix([0,0,0])

    def run_full_process(self):
        self.step_1_kinematics()
        self.step_2_jacobian_M_G()
        self.step_3_coriolis_combined()
        return self.step_4_prepare_export()

    def step_1_kinematics(self):
        T_acc = sp.eye(4)
        R_acc = sp.eye(3)
        omega_acc = sp.Matrix([0, 0, 0])

        for i, (j_type, link_vec) in enumerate(zip(self.joint_config, self.link_vectors_mask)):
            q = dynamicsymbols(f'q{i+1}')
            dq = dynamicsymbols(f'q{i+1}', 1)
            self.q.append(q)
            self.dq.append(dq)
            m = sp.symbols(f'm{i+1}')
            L = sp.symbols(f'L{i+1}')
            self.masses.append(m)
            self.params_list.extend([m, L])

            type_char, axis_char = j_type[0], j_type[1].lower()
            axis_vec_local = self._get_axis_vector(axis_char)

            if type_char == 'R':
                axis_vec_global = R_acc * axis_vec_local
                omega_new = omega_acc + axis_vec_global * dq
                R_j = self._rot_matrix_local(axis_char, q)
                P_j = sp.Matrix([0,0,0])
            elif type_char == 'D':
                omega_new = omega_acc
                R_j = sp.eye(3)
                P_j = sp.Matrix([0,0,0])
                if axis_char == 'x': P_j[0] = q
                if axis_char == 'y': P_j[1] = q
                if axis_char == 'z': P_j[2] = q

            R_acc = R_acc * R_j
            self.rotation_matrices.append(R_acc)
            self.angular_velocities.append(omega_new)
            omega_acc = omega_new

            T_joint = sp.eye(4)
            T_joint[0:3, 0:3] = R_j
            T_joint[0:3, 3] = P_j
            T_at_joint_start = T_acc * T_joint

            P_link_vec = link_vec * L
            T_link = sp.eye(4)
            T_link[0:3, 3] = P_link_vec
            T_acc = T_at_joint_start * T_link
            self.frames.append(T_acc)

            p_start = T_at_joint_start[0:3, 3]
            p_end = T_acc[0:3, 3]
            self.com_positions_global.append((p_start + p_end)/2)

    def step_2_jacobian_M_G(self):
        n = len(self.q)
        self.M = sp.zeros(n, n)
        V_tot = 0
        for i in range(n):
            m = self.masses[i]
            pos_cm = self.com_positions_global[i]
            R_global = self.rotation_matrices[i]
            J_v = pos_cm.jacobian(self.q)
            J_w = self.angular_velocities[i].jacobian(self.dq)

            Ixx, Iyy, Izz = sp.symbols(f'Ixx{i+1} Iyy{i+1} Izz{i+1}')
            I_local = sp.Matrix([[Ixx, 0, 0], [0, Iyy, 0], [0, 0, Izz]])
            I_global = R_global * I_local * R_global.T

            self.M += m * J_v.T * J_v + J_w.T * I_global * J_w
            V_tot += m * self.g * pos_cm[2]

        self.G_vec = sp.Matrix([V_tot]).jacobian(self.q).T
        J_lin = self.frames[-1][0:3, 3].jacobian(self.q)
        J_ang = self.angular_velocities[-1].jacobian(self.dq)
        self.Jacobian = J_lin.col_join(J_ang)

    def step_3_coriolis_combined(self):
        n = len(self.q)
        self.C_total = sp.zeros(n, 1)
        dM_dq = [self.M.diff(qk) for qk in self.q]
        for i in range(n):
            termo_linha = 0
            for j in range(n):
                for k in range(j, n):
                    dM_ij_dk = dM_dq[k][i, j]
                    dM_ik_dj = dM_dq[j][i, k]
                    dM_jk_di = dM_dq[i][j, k]
                    c_ijk = sp.Rational(1,2) * (dM_ij_dk + dM_ik_dj - dM_jk_di)
                    if c_ijk != 0:
                        termo = c_ijk * self.dq[j] * self.dq[k]
                        if k != j: termo *= 2
                        termo_linha += termo
            self.C_total[i] = sp.collect(termo_linha, self.dq)

    def step_4_prepare_export(self):
        mapa_subs = {}
        for i in range(len(self.q)):
            mapa_subs[self.q[i]] = sp.Symbol(f'q{i+1}')
            mapa_subs[self.dq[i]] = sp.Symbol(f'dq{i+1}')

        dq_vec = sp.Matrix(self.dq)
        V_cartesian = self.Jacobian * dq_vec if self.Jacobian is not None else None
        return {
            "M": self.M, "G": self.G_vec, "C": self.C_total, "J": self.Jacobian,
            "FK_Pos": self.frames[-1], "FK_Vel": V_cartesian, "Subs": mapa_subs, "Mode": "Air"
        }

    def step_5_export_cse(self):
        print("\n" + "="*40)
        print(" OTIMIZANDO E EXPORTANDO (CSE) ")
        print("="*40)

        mapa_subs = {}
        for i in range(len(self.q)):
            sym_q = sp.Symbol(f'q{i+1}')
            sym_dq = sp.Symbol(f'dq{i+1}')
            mapa_subs[self.q[i]] = sym_q
            mapa_subs[self.dq[i]] = sym_dq

        # Agrupa expressões
        expressoes = [
            self.M.subs(mapa_subs),
            self.G_vec.subs(mapa_subs),
            self.C_total.subs(mapa_subs),
            self.Jacobian.subs(mapa_subs)
        ]

        print("   Compactando equações (Isso reduz o tamanho do código)...")
        replacements, reduced = sp.cse(expressoes)

        print("% --- Variáveis Auxiliares ---")
        for sym, expr in replacements:
            print(f"{sym} = {octave_code(expr)};")

        M_opt, G_opt, C_opt, J_opt = reduced

        print("\n% --- Matrizes Finais ---")
        print("M = " + octave_code(M_opt) + ";")
        print("G = " + octave_code(G_opt) + ";")
        print("V_total = " + octave_code(C_opt) + "; % Coriolis + Centripeta Juntos")
        print("J = " + octave_code(J_opt) + ";")


# --- CLASSE AVANÇADA (ÁGUA / UVMS) ---
class RobotMathHydro(RobotMathEngine):
    def __init__(self, joint_config, link_vectors_mask):
        super().__init__(joint_config, link_vectors_mask)
        self.rho = sp.symbols('rho')
        self.volumes = []

    def run_full_process(self):
        self.step_1_kinematics()
        self.step_2_jacobian_M_G()
        self.step_3_coriolis_combined()
        return self.step_4_prepare_export()

    def step_1_kinematics(self):
        super().step_1_kinematics()
        for i in range(len(self.q)):
            vol = sp.symbols(f'vol{i+1}')
            self.volumes.append(vol)

    def step_2_jacobian_M_G(self):
        n = len(self.q)
        self.M = sp.zeros(n, n)
        V_tot = 0
        for i in range(n):
            m = self.masses[i]
            vol = self.volumes[i]
            pos_cm = self.com_positions_global[i]
            R_global = self.rotation_matrices[i]
            J_v = pos_cm.jacobian(self.q)
            J_w = self.angular_velocities[i].jacobian(self.dq)

            # Inércia RB
            Ixx, Iyy, Izz = sp.symbols(f'Ixx{i+1} Iyy{i+1} Izz{i+1}')
            I_local_RB = sp.Matrix([[Ixx, 0, 0], [0, Iyy, 0], [0, 0, Izz]])
            I_global_RB = R_global * I_local_RB * R_global.T

            # Massa Adicionada
            ma_u, ma_v, ma_w = sp.symbols(f'ma_u{i+1} ma_v{i+1} ma_w{i+1}')
            ma_p, ma_q, ma_r = sp.symbols(f'ma_p{i+1} ma_q{i+1} ma_r{i+1}')
            MA_lin_global = R_global * sp.Matrix([[ma_u, 0, 0], [0, ma_v, 0], [0, 0, ma_w]]) * R_global.T
            MA_rot_global = R_global * sp.Matrix([[ma_p, 0, 0], [0, ma_q, 0], [0, 0, ma_r]]) * R_global.T

            # M Total = RB + Added
            self.M += J_v.T * (m * sp.eye(3) + MA_lin_global) * J_v
            self.M += J_w.T * (I_global_RB + MA_rot_global) * J_w

            # Potencial = Peso - Empuxo
            peso_aparente = (m - self.rho * vol) * self.g
            V_tot += peso_aparente * pos_cm[2]

        self.G_vec = sp.Matrix([V_tot]).jacobian(self.q).T
        J_lin = self.frames[-1][0:3, 3].jacobian(self.q)
        J_ang = self.angular_velocities[-1].jacobian(self.dq)
        self.Jacobian = J_lin.col_join(J_ang)

    def step_4_prepare_export(self):
        data = super().step_4_prepare_export()
        data["Mode"] = "Hydro"
        return data



# ==============================================================================
# 2. LÓGICA DO GRADIO
# ==============================================================================
def processar_gradio(num_juntas, modo, *args):
    # args contém todos os inputs (12 dropdowns + 12*3 checkboxes)
    # Precisamos reconstruir as listas apenas até num_juntas

    # Cada junta tem 4 componentes: Dropdown, CbX, CbY, CbZ
    # Total inputs no args = 48

    lista_tipos = []
    lista_elos = []

    try:
        num = int(num_juntas)

        for i in range(num):
            base_idx = i * 4
            tipo = args[base_idx]
            vx = 1 if args[base_idx+1] else 0
            vy = 1 if args[base_idx+2] else 0
            vz = 1 if args[base_idx+3] else 0

            lista_tipos.append(tipo)
            lista_elos.append([vx, vy, vz])

        # Seleciona a Engine
        if modo == "Água (UVMS)":
            bot = RobotMathHydro(lista_tipos, lista_elos)
        else:
            bot = RobotMathEngine(lista_tipos, lista_elos)

        # Roda
        res = bot.run_full_process()

        # Salva arquivos
        folder = "Output_Gradio"
        if not os.path.exists(folder): os.makedirs(folder)

        mapa = res["Subs"]
        paths = []

        def save(fname, expr):
            fpath = f"{folder}/{fname}.txt"
            with open(fpath, "w") as f:
                f.write(f"% --- {fname} [{res['Mode']}] ---\n")
                if expr is not None:
                    try:
                        if hasattr(expr, 'subs'): f.write(octave_code(expr.subs(mapa)))
                        else: f.write(str(expr))
                    except: f.write(str(expr))
                else:
                    f.write("% Nao calculado")
            return fpath

        paths.append(save("Matriz_M", res["M"]))
        paths.append(save("Vetor_G", res["G"]))
        paths.append(save("Vetor_C", res["C"]))
        paths.append(save("Cinematica_Posicao", res["FK_Pos"]))
        paths.append(save("Cinematica_Velocidade", res["FK_Vel"]))

        if res["J"] is not None:
            paths.append(save("Jacobiano_Linear", res["J"][0:3, :]))
            paths.append(save("Jacobiano_Angular", res["J"][3:6, :]))
        else:
            paths.append(None); paths.append(None)

        return paths[0], paths[1], paths[2], paths[3], paths[4], paths[5], paths[6], f"Sucesso! Modo: {res['Mode']}"

    except Exception as e:
        return [None]*7 + [f"Erro: {str(e)}"]

# ==============================================================================
# 3. INTERFACE GRADIO
# ==============================================================================
with gr.Blocks(title="Hephaestus Web") as demo:
    gr.Markdown("# 🔱 Hephaestus v2.0 (Web)")
    gr.Markdown("Gerador de Modelagem Simbólica para Robôs e UVMS.")
    gr.Markdown("Escolha **'Água'** para incluir Massa Adicionada e Empuxo nas equações.")

    with gr.Row():
        with gr.Column(scale=1):
            mode_radio = gr.Radio(["Ar (Seco)", "Água (UVMS)"], label="Ambiente", value="Ar (Seco)")
            num_juntas_state = gr.State(value=1)

            # Botões de Controle
            btn_add = gr.Button("+ Adicionar Junta")
            btn_remove = gr.Button("- Remover Junta")

        with gr.Column(scale=3):
            # Container das Juntas (Máximo 12)
            rows = []
            all_inputs = [] # Lista plana para passar à função

            for i in range(12):
                with gr.Row(visible=(i==0)) as r:
                    dd = gr.Dropdown(["Rz", "Ry", "Rx", "Dz", "Dy", "Dx"], label=f"Junta {i+1}", value="Rz", scale=2)
                    cx = gr.Checkbox(label="Elo X", value=False)
                    cy = gr.Checkbox(label="Elo Y", value=False)
                    cz = gr.Checkbox(label="Elo Z", value=(i==0))

                    all_inputs.extend([dd, cx, cy, cz])
                rows.append(r)

    btn_calc = gr.Button("GERAR MODELO 🚀", variant="primary")
    lbl_status = gr.Label(value="Aguardando...", label="Status")

    with gr.Row():
        f1 = gr.File(label="Matriz M")
        f2 = gr.File(label="Vetor G")
        f3 = gr.File(label="Vetor C")
    with gr.Row():
        f4 = gr.File(label="Cinemática Pos")
        f5 = gr.File(label="Cinemática Vel")
    with gr.Row():
        f6 = gr.File(label="Jacobiano Lin")
        f7 = gr.File(label="Jacobiano Ang")

    # --- LÓGICA VISUAL ---
    def update_visibility(curr_num, action):
        new_num = curr_num
        if action == "add" and curr_num < 12: new_num += 1
        if action == "rem" and curr_num > 1: new_num -= 1

        updates = []
        for i in range(12):
            updates.append(gr.Row.update(visible=(i < new_num)))
        return [new_num] + updates

    btn_add.click(lambda n: update_visibility(n, "add"), inputs=[num_juntas_state], outputs=[num_juntas_state] + rows)
    btn_remove.click(lambda n: update_visibility(n, "rem"), inputs=[num_juntas_state], outputs=[num_juntas_state] + rows)

    # --- LÓGICA DE CÁLCULO ---
    btn_calc.click(
        fn=processar_gradio,
        inputs=[num_juntas_state, mode_radio] + all_inputs,
        outputs=[f1, f2, f3, f4, f5, f6, f7, lbl_status]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b5b6bac7804977d918.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
import sympy as sp

# --- CONFIGURAÇÃO DA VALIDAÇÃO ---
# Robô 2 Links Planar (Como na imagem do Niku)
inputs_joints = ['Rz', 'Rz']
inputs_links = [[1, 0, 0], [1, 0, 0]]

# Instancia sua Engine
bot = RobotMathEngine(inputs_joints, inputs_links)

# Ajuste crítico para bater com a imagem do Niku (Robô Vertical):
# A imagem usa cossenos para gravidade, implicando que 0 graus é horizontal e g atua na vertical.
# Vamos forçar a gravidade na direção X negativa para simular isso na validação de G
bot.g = sp.symbols('g')
# Se a sua engine permite definir o vetor gravidade, seria algo como g_vec = [-g, 0, 0]
# Se não, vamos focar na validação de M e V (Coriolis), que independem da gravidade.

bot.step_1_kinematics()
bot.step_2_jacobian_M_G()
bot.step_3_coriolis_combined()
bot.step_5_export_cse()

print("\n" + "="*40)
print("--- VALIDAÇÃO CRUZADA COM NIKU (IMAGEM) ---")
print("="*40)

# 1. Definição das Variáveis Simbólicas do Niku
m1, m2, l1, l2, g = sp.symbols('m1 m2 L1 L2 g')
q1, q2 = bot.q[0], bot.q[1]
dq1, dq2 = bot.dq[0], bot.dq[1]

# Funções trigonométricas compactas (como no livro)
# C1 = cos(q1), C12 = cos(q1+q2)...
C2 = sp.cos(q2)
S2 = sp.sin(q2)
C1 = sp.cos(q1)
C12 = sp.cos(q1 + q2)

# --- 2. MONTANDO AS MATRIZES DA IMAGEM (MANUALMENTE) ---

# A) Matriz de Massa (M) - Niku
M11_n = (sp.Rational(1,3)*m1*l1**2 + m2*l1**2 + sp.Rational(1,3)*m2*l2**2 + m2*l1*l2*C2)
M12_n = (sp.Rational(1,3)*m2*l2**2 + sp.Rational(1,2)*m2*l1*l2*C2)
M21_n = M12_n # Simétrica
M22_n = (sp.Rational(1,3)*m2*l2**2)
M_Niku = sp.Matrix([[M11_n, M12_n], [M21_n, M22_n]])

# B) Matriz de Coriolis e Centrípeta (V) - Niku
# Na imagem, eles estão separados. Vamos escrever igual e somar.

# Termo Centrípeta (multiplica [dq1^2, dq2^2])
Mat_Centripeta = sp.Matrix([
    [0, -sp.Rational(1,2)*m2*l1*l2*S2],
    [sp.Rational(1,2)*m2*l1*l2*S2, 0]
])
Vec_dq_sq = sp.Matrix([dq1**2, dq2**2])
V_cent_Niku = Mat_Centripeta * Vec_dq_sq

# Termo Coriolis (multiplica [dq1*dq2, dq2*dq1])
Mat_Coriolis = sp.Matrix([
    [-(m2*l1*l2*S2), 0],
    [0, 0]
])
Vec_dq_cross = sp.Matrix([dq1*dq2, dq2*dq1])
V_cor_Niku = Mat_Coriolis * Vec_dq_cross

# VETOR TOTAL DO NIKU (Soma dos dois)
V_total_Niku = V_cent_Niku + V_cor_Niku

# C) Vetor Gravidade (G) - Niku
# Imagem: [(0.5*m1+m2)g*l1*C1 + 0.5*m2*g*l2*C12; 0.5*m2*g*l2*C12]
G1_n = (sp.Rational(1,2)*m1 + m2)*g*l1*C1 + sp.Rational(1,2)*m2*g*l2*C12
G2_n = sp.Rational(1,2)*m2*g*l2*C12
G_Niku = sp.Matrix([G1_n, G2_n])


# --- 3. MAPA DE SUBSTITUIÇÃO (Seu Robô -> Livro) ---
# A sua engine usa parâmetros genéricos (cx, Izz). O livro usa hastes finas.
subs_map = {
    # Geometria
    sp.symbols('L1'): l1, sp.symbols('L2'): l2,
    sp.symbols('m1'): m1, sp.symbols('m2'): m2,

    # Centro de Massa (No meio da haste, como assumido pelo Niku 1/3 e 1/12)
    sp.symbols('cx1'): l1/2, sp.symbols('cx2'): l2/2,
    sp.symbols('cy1'): 0, sp.symbols('cz1'): 0,
    sp.symbols('cy2'): 0, sp.symbols('cz2'): 0,

    # Inércias (Haste fina girando pela ponta = 1/3 mL^2, pelo centro = 1/12 mL^2)
    # Sua engine usa Steiner a partir do CM, então usamos I_cm = 1/12 mL^2
    sp.symbols('Izz1'): sp.Rational(1,12)*m1*l1**2,
    sp.symbols('Izz2'): sp.Rational(1,12)*m2*l2**2,

    # Zerar inércias irrelevantes para planar
    sp.symbols('Ixx1'): 0, sp.symbols('Iyy1'): 0,
    sp.symbols('Ixx2'): 0, sp.symbols('Iyy2'): 0
}


# --- 4. COMPARAÇÃO ---

print("\n>>> Comparando MATRIZ DE MASSA (M)...")
Diff_M = sp.simplify(sp.expand(bot.M.subs(subs_map) - M_Niku))
sp.pprint(Diff_M)
if Diff_M == sp.zeros(2,2): print("✅ M: Bateu Perfeito!")
else: print("❌ M: Tem diferença.")

print("\n>>> Comparando CORIOLIS + CENTRÍPETA (V_total)...")
# Nota: bot.C_vec ou bot.C_total dependendo da versão da sua classe
# Assumindo que sua classe salva em bot.C_total ou bot.C_vec
try:
    V_Engine = bot.C_total # Tente C_total (versão otimizada)
except:
    V_Engine = bot.C_vec   # Ou C_vec (versão Christoffel)

Diff_V = sp.simplify(sp.expand(V_Engine.subs(subs_map) - V_total_Niku))
sp.pprint(Diff_V)
if Diff_V == sp.zeros(2,1): print("✅ V (Coriolis+Cent): Bateu Perfeito!")
else: print("❌ V: Tem diferença.")

print("\n>>> Comparando GRAVIDADE (G)...")
# OBS: Isso só vai bater se a sua engine tiver sido configurada
# para gravidade alinhada com o modelo da imagem (vertical).
# Se der erro aqui, é provavelmente apenas a direção do vetor 'g' na sua engine.
G_Engine = bot.G_vec.subs(subs_map)

# Tenta validar G normalizando a direção (ignorando sinal ou eixo se necessário)
# Mas vamos testar direto primeiro
Diff_G = sp.simplify(sp.expand(G_Engine - G_Niku))
# sp.pprint(Diff_G)

if Diff_G == sp.zeros(2,1):
    print("✅ G: Bateu Perfeito!")
else:
    print("⚠️ G: Deu diferença (Esperado se a gravidade da Engine for Z e do Livro for X/Y)")
    print("   Se M e V bateram, sua dinâmica está correta.")


 OTIMIZANDO E EXPORTANDO (CSE) 
   Compactando equações (Isso reduz o tamanho do código)...
% --- Variáveis Auxiliares ---
x0 = sin(q1);
x1 = L1.^2.*m1/4;
x2 = cos(q1);
x3 = L1.*x2;
x4 = sin(q2);
x5 = x0.*x4;
x6 = cos(q2);
x7 = -x2.*x6 + x5;
x8 = -x7;
x9 = L2.*x8;
x10 = x9/2;
x11 = x10 + x3;
x12 = L1.*x0;
x13 = x0.*x6;
x14 = x2.*x4;
x15 = -x13 - x14;
x16 = L2.*x15/2 - x12;
x17 = m2.*x11;
x18 = L2.*x15;
x19 = x18/2;
x20 = m2.*x16;
x21 = Izz2 + x10.*x17 + x19.*x20;
x22 = L2.^2.*m2;
x23 = x22/4;
x24 = L2.*x20.*x7/2 + x17.*x19;
x25 = 2*dq1.*dq2;
x26 = dq1.^2;
x27 = -L2.*x15;
x28 = m2/2;
x29 = dq2.^2;
x30 = x22/8;
x31 = x30.*x8.*(-2*x13 - 2*x14);
x32 = x15.*x30.*(-2*x2.*x6 + 2*x5);
x33 = x15.*x23;
x34 = x31 + x32;

% --- Matrizes Finais ---
M = [Izz1 + Izz2 + m2.*x11.^2 + m2.*x16.^2 + x0.^2.*x1 + x1.*x2.^2 x21; x21 Izz2 + x15.^2.*x23 + x23.*x8.^2];
G = [0; 0];
V_total = [x24.*x25 + x26.*(x11.*x28.*(-2*x12 - x27) + x16.*x28.*(L2.*x7 - 2*x3)) + x29.*(x24 - x31 - x32 + x33.*x7 + x33.*x8); x25